In [1]:
import pandas as pd
import numpy as np

In [2]:
test = pd.read_csv('test_final_100k.csv')

A continuación vamos a realizar las mismas operaciones con los dataframes de postulantes y avisos que realizamos en el notebook principal, a fin de tener un dataframe test con los features correspondientes para que los algoritmos de Machine Learning puedan predecir los resultados.

No realizamos comentarios respecto a las operaciones ya que estos se encuentran en el notebook principal.

## Información de los postulantes

### DF 1: Educación de los postulantes

In [3]:
postulantes_educ = pd.read_csv('datos_navent_fiuba/h15_fiuba_1_postulantes_educacion.csv')
postulantes_educ_2 = pd.read_csv('datos_navent_fiuba/d15_fiuba_1_postulantes_educacion.csv')

postulantes_educ = pd.merge(postulantes_educ, postulantes_educ_2, on=['idpostulante', 'nombre', 'estado']\
                       , how='outer')

postulantes_educ_2 = pd.read_csv('datos_navent_fiuba/fiuba_1_postulantes_educacion.csv')

postulantes_educ = pd.merge(postulantes_educ, postulantes_educ_2, on=['idpostulante', 'nombre', 'estado']\
                       , how='outer')

postulantes_educ.rename(columns={'nombre':'titulo_univ'}, inplace=True)

postulantes_educ.loc[(postulantes_educ["titulo_univ"] == "Otro")] = 1
 
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Secundario')&\
                     (postulantes_educ['estado'] == 'Abandonado'),'orden_estudio'] = 2

postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Secundario')&\
                     (postulantes_educ['estado'] == 'En Curso'),'orden_estudio'] = 3

postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Secundario')&\
                     (postulantes_educ['estado'] == 'Graduado'),'orden_estudio'] = 4

postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Terciario/Técnico')&\
                     (postulantes_educ['estado'] == 'Abandonado'),'orden_estudio'] = 4

postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Terciario/Técnico')&\
                     (postulantes_educ['estado'] == 'En Curso'),'orden_estudio'] = 6

postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Terciario/Técnico')&\
                     (postulantes_educ['estado'] == 'Graduado'),'orden_estudio'] = 7

postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Universitario')&\
                     (postulantes_educ['estado'] == 'Abandonado'),'orden_estudio'] = 8

postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Universitario')&\
                     (postulantes_educ['estado'] == 'En Curso'),'orden_estudio'] = 9

postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Universitario')&\
                     (postulantes_educ['estado'] == 'Graduado'),'orden_estudio'] = 10

postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Posgrado')&\
                     (postulantes_educ['estado'] == 'Abandonado'),'orden_estudio'] = 11

postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Posgrado')&\
                     (postulantes_educ['estado'] == 'En Curso'),'orden_estudio'] = 12

postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Posgrado')&\
                     (postulantes_educ['estado'] == 'Graduado'),'orden_estudio'] = 13

postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Master')&\
                     (postulantes_educ['estado'] == 'Abandonado'),'orden_estudio'] = 14

postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Master')&\
                     (postulantes_educ['estado'] == 'En Curso'),'orden_estudio'] = 15

postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Master')&\
                     (postulantes_educ['estado'] == 'Graduado'),'orden_estudio'] = 16

postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Doctorado')&\
                     (postulantes_educ['estado'] == 'Abandonado'),'orden_estudio'] = 17

postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Doctorado')&\
                     (postulantes_educ['estado'] == 'En Curso'),'orden_estudio'] = 18

postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Doctorado')&\
                     (postulantes_educ['estado'] == 'Graduado'),'orden_estudio'] = 19

postulantes_educ = postulantes_educ.dropna()

postulantes_educ = postulantes_educ.sort_values('orden_estudio', ascending=False)

postulantes_educ = postulantes_educ.drop_duplicates('idpostulante')

postulantes_educ.drop(columns={'titulo_univ', 'estado'}, axis=1, inplace=True)

postulantes_educ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 444275 entries, 344204 to 97377
Data columns (total 2 columns):
idpostulante     444275 non-null object
orden_estudio    444275 non-null float64
dtypes: float64(1), object(1)
memory usage: 10.2+ MB


### DF 2: Género y Edad de los postulantes

In [4]:
postulantes = pd.read_csv('datos_navent_fiuba/h15_fiuba_2_postulantes_genero_y_edad.csv')
postulantes_gen_nac_2 = pd.read_csv('datos_navent_fiuba/d15_fiuba_2_postulantes_genero_y_edad.csv')

postulantes = pd.merge(postulantes, postulantes_gen_nac_2, on=['idpostulante', 'fechanacimiento', 'sexo']\
                       , how='outer')

postulantes_gen_nac_2 = pd.read_csv('datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')

postulantes = pd.merge(postulantes, postulantes_gen_nac_2, on=['idpostulante', 'fechanacimiento', 'sexo']\
                       , how='outer')

postulantes = postulantes[postulantes['sexo'] != '0.0']

postulantes = postulantes.sort_values('sexo', ascending=True)

postulantes = postulantes.drop_duplicates('idpostulante')

postulantes = postulantes.rename(columns={'fechanacimiento': 'edad'})

fecha_nac = postulantes['edad'].str.split('-')
año_nac = fecha_nac.str[0]
postulantes['edad'] = año_nac
postulantes['edad'] = pd.to_numeric(postulantes['edad'], errors='coerce').fillna(0).astype(np.int64)

edad_min = postulantes['edad'] > 2000
edad_max = postulantes['edad'] < 1940
condicion_final = ((edad_min | edad_max))
postulantes = postulantes[np.logical_not(condicion_final)]

postulantes['edad'] = postulantes['edad'].apply(lambda x: 2018-x)

postulantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 478235 entries, 165770 to 147639
Data columns (total 3 columns):
idpostulante    478235 non-null object
edad            478235 non-null int64
sexo            478235 non-null object
dtypes: int64(1), object(2)
memory usage: 14.6+ MB


### Merge de los DF 1 y 2

In [5]:
postulantes = pd.merge(postulantes_educ, postulantes, on='idpostulante'\
                       , how='outer')

postulantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 487381 entries, 0 to 487380
Data columns (total 4 columns):
idpostulante     487381 non-null object
orden_estudio    444275 non-null float64
edad             478235 non-null float64
sexo             478235 non-null object
dtypes: float64(2), object(2)
memory usage: 18.6+ MB


In [6]:
postulantes.loc[(postulantes['sexo'] == 'FEM'),'sexo'] = 1

postulantes.loc[(postulantes['sexo'] == 'MASC'),'sexo'] = 2

postulantes.loc[(postulantes['orden_estudio'].isnull()), 'orden_estudio'] = 0
postulantes.loc[(postulantes['edad'].isnull()), 'edad'] = 0
postulantes.loc[(postulantes['sexo']=='NO_DECLARA'), 'sexo'] = 0
postulantes.loc[(postulantes['sexo'].isnull()), 'sexo'] = 0

postulantes.head()

,idpostulante,orden_estudio,edad,sexo
0,16weJ3,19.0,57.0,2
1,Nzr4aX4,19.0,18.0,2
2,rmdarVb,19.0,36.0,2
3,PmVe3RZ,19.0,56.0,2
4,PmGvDM0,19.0,27.0,2


In [7]:
postulantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 487381 entries, 0 to 487380
Data columns (total 4 columns):
idpostulante     487381 non-null object
orden_estudio    487381 non-null float64
edad             487381 non-null float64
sexo             487381 non-null int64
dtypes: float64(2), int64(1), object(1)
memory usage: 18.6+ MB


## Detalles de los avisos

In [8]:
avisos_detalles = pd.read_csv('datos_navent_fiuba/h15_fiuba_6_avisos_detalle.csv')
avisos_detalles_2 = pd.read_csv('datos_navent_fiuba/d15_fiuba_6_avisos_detalle.csv')

avisos_detalles = pd.concat([avisos_detalles, avisos_detalles_2])

avisos_detalles_2 = pd.read_csv('datos_navent_fiuba/fiuba_6_avisos_detalle.csv')

avisos_detalles = pd.concat([avisos_detalles, avisos_detalles_2])

avisos_detalles = avisos_detalles.drop_duplicates(['idaviso'])

avisos_detalles.rename(columns={'denominacion_empresa':'nombre_empresa'}, inplace=True)

avisos_detalles.drop(columns={'idpais', 'ciudad', 'mapacalle'}, axis=1, inplace=True)

avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Full-time'),'tipo_de_trabajo'] = 10
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Part-time'),'tipo_de_trabajo'] = 9
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Teletrabajo'),'tipo_de_trabajo'] = 8
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Por Horas'),'tipo_de_trabajo'] = 7
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Pasantia'),'tipo_de_trabajo'] = 6
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Temporario'),'tipo_de_trabajo'] = 5
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Por Contrato'),'tipo_de_trabajo'] = 4
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Fines de Semana'),'tipo_de_trabajo'] = 3
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Primer empleo'),'tipo_de_trabajo'] = 2
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Voluntario'),'tipo_de_trabajo'] = 1

avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Gerencia / Alta Gerencia / Dirección'),'nivel_laboral'] = 5
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Jefe / Supervisor / Responsable'),'nivel_laboral'] = 4
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Senior / Semi-Senior'),'nivel_laboral'] = 3
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Junior'),'nivel_laboral'] = 2
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Otro'),'nivel_laboral'] = 1

avisos_detalles.head()

,idaviso,titulo,descripcion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,nombre_empresa
0,1111556097,Responsable de Marketing-Producto Div. Constru...,<p>Buscamos un <strong>Responsable de Producto...,Gran Buenos Aires,10,3,Desarrollo de Negocios,Sinteplast S.A.
1,1111949392,Asesor de Servicios - Postventa Chevrolet,"<p><span style="""">Para nuestro cliente, conces...",Gran Buenos Aires,10,3,Ingeniería Eléctrica y Electrónica,Roycan
2,1112145935,Analistas de Impuestos Sr Ssr y Jr,<p>Importante Estudio de Auditoria y Consultor...,Gran Buenos Aires,10,3,Impuestos,Lighthouse Smart
3,1112146010,Software Engineer,<p> </p><p>Chevron is a multinational company ...,Gran Buenos Aires,10,3,Programación,Chevron Buenos Aires Shared Services Center
4,1112211475,Business Developer Manager,"<p>Importante empresa de energías renovables, ...",Gran Buenos Aires,10,3,Comercial,RANDSTAD


In [9]:
avisos_detalles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24950 entries, 0 to 12661
Data columns (total 8 columns):
idaviso            24950 non-null int64
titulo             24950 non-null object
descripcion        24950 non-null object
nombre_zona        24950 non-null object
tipo_de_trabajo    24950 non-null int64
nivel_laboral      24950 non-null int64
nombre_area        24950 non-null object
nombre_empresa     24943 non-null object
dtypes: int64(3), object(5)
memory usage: 1.7+ MB


## Merge del Test File con la información de los postulantes

In [10]:
test_final = pd.merge(test, postulantes, on='idpostulante', how='left')
test_final.head()

,id,idaviso,idpostulante,orden_estudio,edad,sexo
0,0,739260,6M9ZQR,19.0,42.0,1.0
1,1,739260,6v1xdL,7.0,31.0,2.0
2,2,739260,ezRKm9,10.0,36.0,1.0
3,3,758580,1Q35ej,13.0,69.0,2.0
4,4,758580,EAN4J6,9.0,32.0,1.0


In [11]:
test_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 6 columns):
id               100000 non-null int64
idaviso          100000 non-null int64
idpostulante     100000 non-null object
orden_estudio    98164 non-null float64
edad             98164 non-null float64
sexo             98164 non-null float64
dtypes: float64(3), int64(2), object(1)
memory usage: 5.3+ MB


In [12]:
test_final.loc[(test_final['orden_estudio'].isnull()), 'orden_estudio'] = 0
test_final.loc[(test_final['edad'].isnull()), 'edad'] = 0
test_final.loc[(test_final['sexo'].isnull()), 'sexo'] = 0

test_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 6 columns):
id               100000 non-null int64
idaviso          100000 non-null int64
idpostulante     100000 non-null object
orden_estudio    100000 non-null float64
edad             100000 non-null float64
sexo             100000 non-null float64
dtypes: float64(3), int64(2), object(1)
memory usage: 5.3+ MB


## Merge del Test Final con los detalles de los avisos

In [13]:
test_final = pd.merge(test_final, avisos_detalles, on='idaviso', how='left')
test_final.head()

,id,idaviso,idpostulante,orden_estudio,edad,sexo,titulo,descripcion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,nombre_empresa
0,0,739260,6M9ZQR,19.0,42.0,1.0,Asistente Comercial,Nuestro cliente es una empresa nac. ubicada en...,Capital Federal,10.0,4.0,Comercial,BUMERAN.COM ARGENTINA
1,1,739260,6v1xdL,7.0,31.0,2.0,Asistente Comercial,Nuestro cliente es una empresa nac. ubicada en...,Capital Federal,10.0,4.0,Comercial,BUMERAN.COM ARGENTINA
2,2,739260,ezRKm9,10.0,36.0,1.0,Asistente Comercial,Nuestro cliente es una empresa nac. ubicada en...,Capital Federal,10.0,4.0,Comercial,BUMERAN.COM ARGENTINA
3,3,758580,1Q35ej,13.0,69.0,2.0,Oracle Financials Sr. Application Developer,"\r\nTeleTech, the worlwide leader in outsource...",Capital Federal,10.0,1.0,Tecnologia / Sistemas,TeleTech
4,4,758580,EAN4J6,9.0,32.0,1.0,Oracle Financials Sr. Application Developer,"\r\nTeleTech, the worlwide leader in outsource...",Capital Federal,10.0,1.0,Tecnologia / Sistemas,TeleTech


In [14]:
test_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 13 columns):
id                 100000 non-null int64
idaviso            100000 non-null int64
idpostulante       100000 non-null object
orden_estudio      100000 non-null float64
edad               100000 non-null float64
sexo               100000 non-null float64
titulo             95552 non-null object
descripcion        95552 non-null object
nombre_zona        95552 non-null object
tipo_de_trabajo    95552 non-null float64
nivel_laboral      95552 non-null float64
nombre_area        95552 non-null object
nombre_empresa     95540 non-null object
dtypes: float64(5), int64(2), object(6)
memory usage: 10.7+ MB


In [15]:
test_final.loc[(test_final['tipo_de_trabajo'].isnull()), 'tipo_de_trabajo'] = 0
test_final.loc[(test_final['nivel_laboral'].isnull()), 'nivel_laboral'] = 0

test_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 13 columns):
id                 100000 non-null int64
idaviso            100000 non-null int64
idpostulante       100000 non-null object
orden_estudio      100000 non-null float64
edad               100000 non-null float64
sexo               100000 non-null float64
titulo             95552 non-null object
descripcion        95552 non-null object
nombre_zona        95552 non-null object
tipo_de_trabajo    100000 non-null float64
nivel_laboral      100000 non-null float64
nombre_area        95552 non-null object
nombre_empresa     95540 non-null object
dtypes: float64(5), int64(2), object(6)
memory usage: 10.7+ MB


In [16]:
test_final.loc[(test_final['nombre_zona'] == 'Gran Buenos Aires'),'nombre_zona'] = 15
test_final.loc[(test_final['nombre_zona'] == 'Capital Federal'),'nombre_zona'] = 14
test_final.loc[(test_final['nombre_zona'] == 'Buenos Aires (fuera de GBA)'),'nombre_zona'] = 13
test_final.loc[(test_final['nombre_zona'] == 'GBA Oeste'),'nombre_zona'] = 12
test_final.loc[(test_final['nombre_zona'] == 'La Plata'),'nombre_zona'] = 11
test_final.loc[(test_final['nombre_zona'] == 'Ciudad de Mendoza'),'nombre_zona'] = 10
test_final.loc[(test_final['nombre_zona'] == 'Mendoza'),'nombre_zona'] = 9
test_final.loc[(test_final['nombre_zona'] == 'Catamarca'),'nombre_zona'] = 8
test_final.loc[(test_final['nombre_zona'] == 'Rosario'),'nombre_zona'] = 7
test_final.loc[(test_final['nombre_zona'] == 'Santa Cruz'),'nombre_zona'] = 6
test_final.loc[(test_final['nombre_zona'] == 'San Juan'),'nombre_zona'] = 5
test_final.loc[(test_final['nombre_zona'] == 'Cordoba'),'nombre_zona'] = 4
test_final.loc[(test_final['nombre_zona'] == 'Tucuman'),'nombre_zona'] = 3
test_final.loc[(test_final['nombre_zona'] == 'Neuquen'),'nombre_zona'] = 2
test_final.loc[(test_final['nombre_zona'] == 'Santa Fe'),'nombre_zona'] = 1

In [18]:
test_final.loc[(test_final['nombre_zona'].isnull()),'nombre_zona'] = 0
test_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 13 columns):
id                 100000 non-null int64
idaviso            100000 non-null int64
idpostulante       100000 non-null object
orden_estudio      100000 non-null float64
edad               100000 non-null float64
sexo               100000 non-null float64
titulo             95552 non-null object
descripcion        95552 non-null object
nombre_zona        100000 non-null int64
tipo_de_trabajo    100000 non-null float64
nivel_laboral      100000 non-null float64
nombre_area        95552 non-null object
nombre_empresa     95540 non-null object
dtypes: float64(5), int64(3), object(5)
memory usage: 10.7+ MB


## Exportamos a un CSV File el test final

In [19]:
test_final.to_csv('test_final.csv', encoding='utf-8', index=False)